Importing libraries for classification and data cleaning


In [1]:
import os
import seaborn as sns 
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import matplotlib.pyplot as plt
from sklearn import ensemble, model_selection, metrics 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split , cross_val_score , RandomizedSearchCV
import xgboost as xgb

%pylab inline

Populating the interactive namespace from numpy and matplotlib


Adding and printing the dataframe with the skateparks

In [3]:
df=pd.read_csv('/content/skatepark_NY_data - DPR_Skateparks_001.csv')
df.head()

,the_geom,SYSTEM,GISPROPNUM,NAME,Google Rating,BMX_PERMIT,INLINE_SKA,SCOOTER_PE,FEATURES_1,FEATURES_2,FEATURES_3,PARK_SURFA,FEATURE_SU,BOROUGH,SHAPE_area,SHAPE_len
0,MULTIPOLYGON (((-73.91996794995724 40.58884837...,B057-06-SKATEPARK-01,B057,Marine Park Skatepark,4.2,NaN,NaN,NaN,Funbox,Quarter Pipe,Bank Ramp,Concrete,Wood,Brooklyn,6121.923885,337.738978
1,MULTIPOLYGON (((-73.87144701429952 40.86716866...,X002-ZN03-SKATEPARK-01,X002,Bronx Park Skatepark,4.3,NaN,NaN,NaN,Quarter Pipe,Funbox,Bank Ramp,Concrete,Metal,Bronx,6196.325183,319.055415
2,MULTIPOLYGON (((-73.82910897363057 40.82693224...,X183-SKATEPARK-01,X183,Bruckner Playground Skatepark,3.9,NaN,NaN,NaN,Funbox,Quarter Pipe,Bank Ramp,Concrete,Concrete,Bronx,4935.574172,283.919460
3,MULTIPOLYGON (((-73.92460034554938 40.83095313...,X034-SKATEPARK-01,X034,Mullaly Skatepark,4.3,Y,NaN,NaN,Funbox,Half-Pipe,Quarter Pipe,Concrete,Wood,Bronx,16409.661300,540.252203
4,MULTIPOLYGON (((-73.92708768919569 40.82560277...,X348-SKATEPARK-01,X348,River Avenue Skatepark,4.4,Y,Y,Y,Funbox,Handrail,Ledge,Concrete,Concrete,Bronx,16451.685880,529.498909




As we can see there are a lot of categorical values, in fact most of the features are categorical. For simplifying dealing with categoricals we can first copy them in a stand-alone dataframe```



In [4]:
df_cat = df.select_dtypes(include=['object']).copy()
df_cat.columns

Index(['the_geom', 'SYSTEM', 'GISPROPNUM', 'NAME', 'BMX_PERMIT', 'INLINE_SKA',
       'SCOOTER_PE', 'FEATURES_1', 'FEATURES_2', 'FEATURES_3', 'PARK_SURFA',
       'FEATURE_SU', 'BOROUGH'],
      dtype='object')

It's also obvious that a lot of the categorical values lacking data - we'll populate them with the most popular and also logical variant - No data

In [5]:
df_cat[['BMX_PERMIT','INLINE_SKA','SCOOTER_PE']]=df[['BMX_PERMIT','INLINE_SKA','SCOOTER_PE']].replace(np.nan, 'N')
df_cat.head()

,the_geom,SYSTEM,GISPROPNUM,NAME,BMX_PERMIT,INLINE_SKA,SCOOTER_PE,FEATURES_1,FEATURES_2,FEATURES_3,PARK_SURFA,FEATURE_SU,BOROUGH
0,MULTIPOLYGON (((-73.91996794995724 40.58884837...,B057-06-SKATEPARK-01,B057,Marine Park Skatepark,N,N,N,Funbox,Quarter Pipe,Bank Ramp,Concrete,Wood,Brooklyn
1,MULTIPOLYGON (((-73.87144701429952 40.86716866...,X002-ZN03-SKATEPARK-01,X002,Bronx Park Skatepark,N,N,N,Quarter Pipe,Funbox,Bank Ramp,Concrete,Metal,Bronx
2,MULTIPOLYGON (((-73.82910897363057 40.82693224...,X183-SKATEPARK-01,X183,Bruckner Playground Skatepark,N,N,N,Funbox,Quarter Pipe,Bank Ramp,Concrete,Concrete,Bronx
3,MULTIPOLYGON (((-73.92460034554938 40.83095313...,X034-SKATEPARK-01,X034,Mullaly Skatepark,Y,N,N,Funbox,Half-Pipe,Quarter Pipe,Concrete,Wood,Bronx
4,MULTIPOLYGON (((-73.92708768919569 40.82560277...,X348-SKATEPARK-01,X348,River Avenue Skatepark,Y,Y,Y,Funbox,Handrail,Ledge,Concrete,Concrete,Bronx


We also don't need all of the categorical columns, so we'll save all the needed columns in the list and filter only columns belonging to that list

In [6]:
cat_columns=['BMX_PERMIT', 'INLINE_SKA',
       'SCOOTER_PE', 'FEATURES_1', 'FEATURES_2', 'FEATURES_3', 'PARK_SURFA',
       'FEATURE_SU', 'BOROUGH']
df_cat=df_cat[cat_columns]
df_cat.head()

,BMX_PERMIT,INLINE_SKA,SCOOTER_PE,FEATURES_1,FEATURES_2,FEATURES_3,PARK_SURFA,FEATURE_SU,BOROUGH
0,N,N,N,Funbox,Quarter Pipe,Bank Ramp,Concrete,Wood,Brooklyn
1,N,N,N,Quarter Pipe,Funbox,Bank Ramp,Concrete,Metal,Bronx
2,N,N,N,Funbox,Quarter Pipe,Bank Ramp,Concrete,Concrete,Bronx
3,Y,N,N,Funbox,Half-Pipe,Quarter Pipe,Concrete,Wood,Bronx
4,Y,Y,Y,Funbox,Handrail,Ledge,Concrete,Concrete,Bronx


So now we have a good set of categorical columns with all data present everywhere. However, categorical values won't work in the model, so we'll need to transform them to numerical values. For that I'm going to use LabelEncoder module of preprocessing library

In [7]:
df_encoded=df_cat.apply(preprocessing.LabelEncoder().fit_transform)

Ok, so it seems that we have quite a good and clean set of Categorical type columns now. That's great! Now lets deal with the rest of the columns. But before we proceed I noticed an interesting thing that I wanted to action - "GEOM" column contains lattituds and lognitudes values that could be beneficial for features engineering. We need to process and save them in a stand alone list - in order to add to a resulting dataframe later 

In [8]:
import re

lattitudes=[]
longitudes=[]
for x in df['the_geom']:
  text=x.replace('MULTIPOLYGON (((','MULTIPOLYGON')
  MATCH = re.search('MULTIPOLYGON(.+?), -', text)
  if MATCH:
    result_string = MATCH.group(1)
    lat=(result_string.rpartition(' ')[0])
    lon=(result_string.rpartition(' ')[2])
    lattitudes.append(float(lat[:7]))
    longitudes.append(float(lon[:6]))


Adding new geo location columns to the dataframe

In [9]:
df['lattitudes']=lattitudes
df['longitudes']=longitudes

All right, now let's have a look at the rest of the columns. I shortlisted attributes we really need and made a sub-dataframe of them which I called df_non_cat

In [10]:
df_non_cat=df[['NAME','Google Rating','SHAPE_area','SHAPE_len','lattitudes','longitudes']]

Now let's concatenate categorical and non-cat dataframe to get the resulting dataset

In [11]:
result_df=pd.concat([df_non_cat, df_encoded], axis=1, join='inner')


Voila! Now we have a clean and ready to process dataframe that we can use for a ML experiment


In [12]:
result_df.head()

,NAME,Google Rating,SHAPE_area,SHAPE_len,lattitudes,longitudes,BMX_PERMIT,INLINE_SKA,SCOOTER_PE,FEATURES_1,FEATURES_2,FEATURES_3,PARK_SURFA,FEATURE_SU,BOROUGH
0,Marine Park Skatepark,4.2,6121.923885,337.738978,-73.919,40.588,0,0,0,1,5,0,1,2,1
1,Bronx Park Skatepark,4.3,6196.325183,319.055415,-73.871,40.867,0,0,0,5,1,0,1,1,0
2,Bruckner Playground Skatepark,3.9,4935.574172,283.919460,-73.829,40.826,0,0,0,1,5,0,1,0,0
3,Mullaly Skatepark,4.3,16409.661300,540.252203,-73.924,40.830,1,0,0,1,2,5,1,2,0
4,River Avenue Skatepark,4.4,16451.685880,529.498909,-73.927,40.825,1,1,1,1,3,4,1,0,0


## Simple EDA with Folium and Plotly

Let's put our findins on the map! Folumn library provides an option to location of parks with the features in buble pop ups

In [13]:
# create map of New york skateparks using latitude and longitude values
import folium
map_ny_parks = folium.Map(location=[40.825,-73.927],zoom_start=7)

for lat, lng, label in zip(result_df['longitudes'], result_df['lattitudes'], df['FEATURES_1']):
    
    label_holder = folium.Popup(folium.IFrame(label, width=100, height=100), max_width=150)
    folium.CircleMarker(
        [lat, lng],
        radius=12,
        popup=label_holder,
        color='red',
        icon_color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny_parks)  

In [14]:
map_ny_parks

Let's also look at how we can perform a simple EDA on core features. First of all we are interested in if having enough features helps get higher rating. for that I'm using Plotly Express library

In [27]:
import plotly.express as px
# df = px.data.tips()
fig = px.histogram(df, x="NAME", y="Google Rating", color="FEATURES_1",
                   marginal="box", 
                   hover_data=df.columns)
fig.show()

The histogram chart doesn't really give a good overview of whats going on as it's not veary readable and we have too many features. 


In [37]:

fig = px.histogram(df, x="NAME", y="Google Rating", color="PARK_SURFA",
                   marginal="box", 
                   hover_data=df.columns)
fig.show()

Box charts give slightly better overview, but we face the issue of "wide" dataframe - meaning that "FEATURE"s distributed accross 3 columns - there fore we need to create several charts to get a good understanding on value distribution.

In [17]:
import plotly.express as px
fig = px.box(df, x="FEATURES_1", y="Google Rating", points='all')
fig.show()

This can be solved if we 'melt' dataframe - that means 'unpivoting' it to a narrower formats. likely there is a built - in pandas function for that 

In [31]:
df_melted=pd.melt(df, id_vars=['NAME'], value_vars=['FEATURES_1','FEATURES_2','FEATURES_3'])


In [33]:
df_melted.sort_values(by=['NAME']).head()

,NAME,variable,value
32,12th & A/Open Road Park,FEATURES_2,Ledge
12,12th & A/Open Road Park,FEATURES_1,Ledge
52,12th & A/Open Road Park,FEATURES_3,Quarter Pipe
29,Astoria Park Skatepark,FEATURES_2,Bank Ramp
9,Astoria Park Skatepark,FEATURES_1,Ledge


In [48]:
melted_df_with_rating=df_melted.merge(result_df, on='NAME', how='left')

Now that we have nice and clean melted database we can build some more sophisticated boxplots like the ones below

In [51]:
fig = px.box(melted_df_with_rating, y="Google Rating", color="value", facet_col="value", 
              points='all', boxmode='overlay')
fig.show()

Lets do the same for "permit" columns - now they re distributed accorss 3 attirbutes 

In [71]:
df_melted_permit=pd.melt(df, id_vars=['NAME'], value_vars=['BMX_PERMIT','INLINE_SKA','SCOOTER_PE'])
df_melted_permit=df_melted_permit.merge(result_df, on='NAME', how='left')


In [72]:
df_melted_permit.head()

,NAME,variable,value,Google Rating,SHAPE_area,SHAPE_len,lattitudes,longitudes,BMX_PERMIT,INLINE_SKA,SCOOTER_PE,FEATURES_1,FEATURES_2,FEATURES_3,PARK_SURFA,FEATURE_SU,BOROUGH
0,Marine Park Skatepark,BMX_PERMIT,NaN,4.2,6121.923885,337.738978,-73.919,40.588,0,0,0,1,5,0,1,2,1
1,Bronx Park Skatepark,BMX_PERMIT,NaN,4.3,6196.325183,319.055415,-73.871,40.867,0,0,0,5,1,0,1,1,0
2,Bruckner Playground Skatepark,BMX_PERMIT,NaN,3.9,4935.574172,283.919460,-73.829,40.826,0,0,0,1,5,0,1,0,0
3,Mullaly Skatepark,BMX_PERMIT,Y,4.3,16409.661300,540.252203,-73.924,40.830,1,0,0,1,2,5,1,2,0
4,River Avenue Skatepark,BMX_PERMIT,Y,4.4,16451.685880,529.498909,-73.927,40.825,1,1,1,1,3,4,1,0,0


In [73]:
df_melted_permit['value']=df_melted_permit['value'].replace(np.nan, 'N')

In [74]:
import plotly.express as px
fig = px.box(df_melted_permit, x="variable", y="Google Rating", color="value",
             notched=True, # used notched shape
             hover_data=["NAME"], points='all' # add day column to hover data
            )
fig.show()

Finally let's see how continuos features correlate with each other. For that we'll be using a scatterplot. I thought it would be intersting to also introduce a new feature that counts a number of permits for every sktepark

In [81]:
df['number_of_permits']=result_df['BMX_PERMIT']+result_df['INLINE_SKA']+result_df['SCOOTER_PE']

In [83]:
import plotly.express as px
fig = px.scatter_matrix(df, dimensions=["SHAPE_area", "SHAPE_len", "Google Rating", "number_of_permits"],
)
fig.show()

However we see that neither skatepark area no number of permits correlate with the rating